In [2]:
import os
import sys
import json
import math
import cachetools
import numpy as np
import pandas as pd
import configparser
from snowflake.snowpark import Session
from copy import copy
from snowflake.snowpark import Row
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col, lit, sql_expr, get, get_path, udf, udtf, table_function, sproc, seq8, uniform, when_matched, when_not_matched, cast, try_cast, asc, asc_nulls_first, asc_nulls_last, collate, startswith, endswith, equal_nan, is_null, in_, when
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType, PandasSeries, PandasSeriesType, PandasDataFrame, PandasDataFrameType
from snowflake.snowpark.exceptions import SnowparkJoinException, SnowparkSQLException
from snowflake.snowpark.files import SnowflakeFile
from snowflake.snowpark.column import METADATA_FILENAME, METADATA_FILE_ROW_NUMBER

# Read snowflake credentials securely
config = configparser.ConfigParser()
config.read('assets/credentials.cfg')

connection_parameters = dict(
   account   =  config['SNOWPARKAWS']['SNOWFLAKE_ACCOUNT'],
   user      =  config['SNOWPARKAWS']['SNOWFLAKE_USER'],
   password  =  config['SNOWPARKAWS']['SNOWFLAKE_PASSWORD'],
   role      =  config['SNOWPARKAWS']['SNOWFLAKE_ROLE'],  # optional
   warehouse =  config['SNOWPARKAWS']['SNOWFLAKE_WAREHOUSE'],  # optional
   database  =  config['SNOWPARKAWS']['SNOWFLAKE_DATABASE'],  # optional
   schema    =  config['SNOWPARKAWS']['SNOWFLAKE_SCHEMA'],  # optional
)

# Pass this dictionary to the Session.builder.configs method to return a builder object that has these connection parameters.
# Call the create method of the builder to establish the session.
session = Session.builder.configs(connection_parameters).create()

#### `Snowflake Stored Procedures`

snowflake.snowpark.stored_procedure.StoredProcedure encapsulates a user defined lambda or function that is returned by [sproc()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.functions.sproc.html#snowflake.snowpark.functions.sproc), [StoredProcedureRegistration.register()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register.html#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register) or [StoredProcedureRegistration.register_from_file()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register_from_file.html#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register_from_file). The constructor of this class is not supposed to be called directly.


### StoredProcedureRegistration 

snowflake.snowpark.stored_procedure.StoredProcedureRegistration provides methods to register lambdas and functions as stored procedures in the Snowflake database. 

- ***session.sproc*** returns an object of this class. You can use this object to register stored procedures that you plan to use in the current session or permanently. The methods that register a stored procedure return a StoredProcedure object.
- **The first parameter of your function should be a snowpark Session**. Also, you need to add snowflake-snowpark-python package (version >= 0.4.0) to your session before trying to create a stored procedure.

#### Ways to Register Stored Procedure

- Use sproc() or [register()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register).
- Use register_from_file().

Additional Reference Codes for registering stored procedures using Snowflake snowpark API - [here](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration)


---------------------------------------

### Snowflake Snowpark supported data types for the parameters

Snowflake supports the following [data types](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration) for the parameters for a stored procedure:


| Python Type | Snowpark Type | SQL Type | 
|--|--|--|
| int |  LongType | NUMBER | 
| decimal.Decimal | DecimalType | NUMBER |
| float | FloatType | FLOAT |
| str | StringType | STRING |
| bool | BooleanType | BOOL |
| datetime.time | TimeType | TIME |
| datetime.date | DateType | DATE |
| datetime.datetime | TimestampType | TIMESTAMP |
| bytes or bytearray | BinaryType | BINARY |
| list | ArrayType | ARRAY |
| dict | MapType | OBJECT |
| Dynamically mapped to the native Python type | VariantType | VARIANT |
| dict | GeographyType | GEOGRAPHY |


- [python/latest/stored_procedures](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/stored_procedures)

In [11]:
x,y = 5,5
session.sql(f"select {x}+{y}").collect()

[Row(5+5=10)]

In [9]:
session.sql(f"select {x}+{y}").collect()[0]

Row(5+5=10)

In [10]:
session.sql(f"select {x}+{y}").collect()[0][0]

10

In [24]:
# functions.sproc: Registers a Python function as a Snowflake Python stored procedure and returns the stored procedure.
import snowflake.snowpark
from snowflake.snowpark.functions import sproc, col
from snowflake.snowpark.types import StringType, BooleanType, IntegerType
#session.add_packages('snowflake-snowpark-python')

@sproc(return_type=IntegerType(), input_types=[IntegerType(), IntegerType()], packages=["snowflake-snowpark-python"], replace=True)
def add_sp(session, x, y):
    return session.sql(f"select {x}+{y}").collect()[0][0]


In [25]:
# Call stored procedure
add_sp(5,5)

10

In [2]:
# @sproc decorator allows you to create temporary stored procedure which you can use within the session
import snowflake.snowpark
from snowflake.snowpark.functions import sproc, col
from snowflake.snowpark.types import StringType, BooleanType, IntegerType
session.add_packages('snowflake-snowpark-python')

@sproc
def refresh_query_history(session: snowflake.snowpark.Session) -> str:
    tables_df = session.sql("show tables in snowpark_db.public")
    tables_df.write.save_as_table(table_name="snowpark_db_public_schema_tables", mode="overwrite")
    return "Success"

In [3]:
refresh_query_history()

'Success'

In [36]:
session.table("snowpark_db_public_schema_tables").select(col('"database_name"'), col('"schema_name"'), col('"name"')).show()

----------------------------------------------------------
|"database_name"  |"schema_name"  |"name"                |
----------------------------------------------------------
|SNOWPARK_DB      |PUBLIC         |10tablename           |
|SNOWPARK_DB      |PUBLIC         |CAR_SALES             |
|SNOWPARK_DB      |PUBLIC         |QUOTED                |
|SNOWPARK_DB      |PUBLIC         |SALES_TEST            |
|SNOWPARK_DB      |PUBLIC         |SAMPLE_PRODUCT_DATA   |
|SNOWPARK_DB      |PUBLIC         |SAMPLE_PRODUCT_DATA1  |
|SNOWPARK_DB      |PUBLIC         |STOCK_PRICE_HISTORY   |
----------------------------------------------------------



In [5]:
# Registering Stored Procedure using Session.sproc.register() method. It allows you to create permanent as well as temporary sprocs.
# For permanent stored procedures, you would require a stage to be mentioned while calling the method
import snowflake.snowpark
from snowflake.snowpark.functions import sproc, col
from snowflake.snowpark.types import StringType, BooleanType, IntegerType
session.add_packages('snowflake-snowpark-python')

def refresh_login_history(session: snowflake.snowpark.Session, status:bool) -> str:
    if (status):
        login_df = session.table('SNOWFLAKE.ACCOUNT_USAGE.LOGIN_HISTORY')
        login_df.write.save_as_table(table_name="loginhistory", mode="overwrite")
        return 'refresh ran successfully'
    return 'table not refreshed'
  
session.sproc.register(name='refresh_login_hist_sp',
                       func=refresh_login_history,
                       return_type=StringType(),
                       input_types=[BooleanType()],
                       replace=True,
                       is_permanent=True,
                       stage_location='@SF_INT_STG',
                       execute_as = 'CALLER'
)

In [6]:
session.call('refresh_login_hist_sp',True)

'refresh ran successfully'

In [8]:
session.table("loginhistory").select(col("USER_NAME"), col("EVENT_TYPE"), col("IS_SUCCESS"), col("ERROR_MESSAGE")).show()

---------------------------------------------------------------
|"USER_NAME"  |"EVENT_TYPE"  |"IS_SUCCESS"  |"ERROR_MESSAGE"  |
---------------------------------------------------------------
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
---------------------------------------------------------------



- [sproc.register_from_file](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration.register_from_file) : Registers a Python function as a Snowflake Python UDF from a Python or zip file, and returns the UDF.

In [18]:
# Registering Stored Procedure using Session.sproc.register_from_file() method. It allows you to register a python function from an external file or from a stage file
# Source script path: assets/scripts/refresh_login_history.py
session.sproc.register_from_file(file_path="scripts/refresh_login_history.py"
                                ,func_name="refresh_login_history"
                                ,return_type=StringType()
                                ,input_types=[BooleanType()]
                                ,name="loginhistory_sp"
                                ,is_permanent=True
                                ,stage_location='@SF_INT_STG'
                                ,replace=True
)

In [19]:
session.call('loginhistory_sp',True)

'refresh ran successfully'

In [20]:
session.table("loginhistory").select(col("USER_NAME"), col("EVENT_TYPE"), col("IS_SUCCESS"), col("ERROR_MESSAGE")).show()

---------------------------------------------------------------
|"USER_NAME"  |"EVENT_TYPE"  |"IS_SUCCESS"  |"ERROR_MESSAGE"  |
---------------------------------------------------------------
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
|PLOGANATHAN  |LOGIN         |YES           |NULL             |
---------------------------------------------------------------



#### `Snowflake Snowpark User Defined Functions`

snowflake.snowpark.udf.UDFRegistration provides methods to register lambdas and functions as UDFs in the Snowflake database.

[session.udf](https://docs.snowflake.com/developer-guide/snowpark/reference/python/api/snowflake.snowpark.Session.udf.html#snowflake.snowpark.Session.udf) returns an object of this class. You can use this object to register UDFs that you plan to use in the current session or permanently. The methods that register a UDF return a UserDefinedFunction object, which you can also use in Column expressions.

Before creating a UDF, think about whether you want to create a vectorized UDF (also referred to as Python UDF Batch API) or a regular UDF. The advantages of a vectorized UDF are:
- The potential for better performance if your Python code operates efficiently on batches of rows.
- Less transformation logic is required if you are calling into libraries that operate on Pandas DataFrames or Pandas arrays.

#### Ways to Register User Defined Stored Procedure

- Use [udf()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.functions.udf#snowflake.snowpark.functions.udf) or [register()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration.register#snowflake.snowpark.udf.UDFRegistration.register).
- Use [register_from_file()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration.register_from_file#snowflake.snowpark.udf.UDFRegistration.register_from_file).

Additional Reference Codes for registering UDFs using Snowflake snowpark API - [here](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration)

- [UDF](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/udf): User-defined functions (UDFs) in Snowpark.

In [21]:
session.range(1,8,2).show()

--------
|"ID"  |
--------
|1     |
|3     |
|5     |
|7     |
--------



In [22]:
# Create a temporary UDF from a lambda and apply it to a dataframe:
add_one_udf = udf(lambda x:x+1, return_type=IntegerType(), input_types=[IntegerType()])

In [24]:
session.range(1,8,2).select(col("ID"), add_one_udf(col("ID")).alias("ID_")).show()

----------------
|"ID"  |"ID_"  |
----------------
|3     |4      |
|7     |8      |
|1     |2      |
|5     |6      |
----------------



In [27]:
# Create a UDF with type hints and @udf decorator and apply it to a dataframe:
@udf
def add_udf(x:int, y:int) -> int:
    return x + y

df = session.create_dataframe([[1,2],[3,4]], schema=["a","b"])
df.show()

#df.select(col("a"), col("b"), add_udf(col("a"),col("b")).alias("c")).show()
df.select(col("a"), col("b"), add_udf("a","b").alias("c")).show()


-------------
|"A"  |"B"  |
-------------
|1    |2    |
|3    |4    |
-------------

-------------------
|"A"  |"B"  |"C"  |
-------------------
|1    |2    |3    |
|3    |4    |7    |
-------------------



In [29]:
# Create a permanent UDF with a name and call it in SQL:
_ = session.udf.register(lambda x,y:x*y
                     ,return_type=IntegerType()
                     ,input_types=[IntegerType(), IntegerType()]
                     ,is_permanent=True
                     ,name="mul"
                     ,replace=True # overwrite udf definition when it already exists
                     ,stage_location="@SF_UDF_INT_STG"
                     )

In [32]:
session.sql("select mul(5,5) as res").show()

---------
|"RES"  |
---------
|25     |
---------



In [34]:
# skip udf creation if it already exists
_ = session.udf.register(lambda x,y:x*y
                     ,return_type=IntegerType()
                     ,input_types=[IntegerType(), IntegerType()]
                     ,is_permanent=True
                     ,name="mul"
                     #,replace=True # Should be comment out
                     ,stage_location="@SF_UDF_INT_STG"
                     ,if_not_exists=True # skip udf creation if it already exists
                     )

`Create a UDF with UDF-level imports and apply it to a dataframe:`

In [38]:
# from <directoryname>.<filename> import <functionname>
from scripts.sf_credit import credit_to_dollar

@udf(imports=[('scripts/sf_credit.py','scripts/sf_credit')])
def wh_credit_calc(wh_size:str, credit:int) -> float:
    if wh_size == 'XS':
        return credit_to_dollar(credit)*1
    elif  wh_size == 'S':
        return credit_to_dollar(credit)*2
    elif  wh_size == 'M':
        return credit_to_dollar(credit)*4
    elif  wh_size == 'L':
        return credit_to_dollar(credit)*8
    elif  wh_size == 'XL':
        return credit_to_dollar(credit)*16
    elif  wh_size == 'XXL':
        return credit_to_dollar(credit)*32
    else:
        return credit_to_dollar(credit)


In [40]:
schema = StructType([StructField('warehouse',StringType()),StructField('credit',IntegerType())])
dataset = [['XS',23],['L',66],['M',11]]
df = session.createDataFrame(dataset,schema= schema)

df.select(col("warehouse"), col("credit"), wh_credit_calc(col("warehouse"), col("credit")).alias("res")).show()

-----------------------------------------------
|"WAREHOUSE"  |"CREDIT"  |"RES"               |
-----------------------------------------------
|L            |66        |1108.8              |
|M            |11        |92.4                |
|XS           |23        |48.300000000000004  |
-----------------------------------------------



In [41]:
# from <directoryname>.<filename> import <functionname>
from scripts.mod5 import mod5

@udf(imports=[('scripts/mod5.py','scripts/mod5')])
def mod5_and_plus1_udf(x: int) -> int:
    return mod5(x) + 1

In [42]:
session.range(1, 8, 2).select(mod5_and_plus1_udf("id")).to_df("col1").collect()

[Row(COL1=2), Row(COL1=4), Row(COL1=1), Row(COL1=3)]

`Create a UDF with UDF-level packages and apply it to a dataframe:`

In [43]:
import numpy as np 
import math 

@udf(packages=["numpy"])
def sin_udf(x: float) -> float:
    return np.sin(x)

In [45]:
df = session.create_dataframe([0.0, 0.5*math.pi], schema=["d"])
df.show()

df.select(sin_udf("d")).to_df("col1").show()

----------------------
|"D"                 |
----------------------
|0.0                 |
|1.5707963267948966  |
----------------------

----------
|"COL1"  |
----------
|1.0     |
|0.0     |
----------



`Creating a UDF from a local Python file:`

In [46]:
mod5_udf = session.udf.register_from_file(
     file_path="scripts/mod5.py"
    ,func_name="mod5"
)

In [48]:
session.range(1, 8, 2).select(col("id"), mod5_udf("id")).show()

-------------------------------------------------------------
|"ID"  |"""SNOWPARK_DB"".""PUBLIC"".SNOWPARK_TEMP_FUNCT...  |
-------------------------------------------------------------
|5     |0                                                   |
|1     |1                                                   |
|7     |2                                                   |
|3     |3                                                   |
-------------------------------------------------------------



`Creating a UDF from a Python file on an internal stage:`

In [53]:
session.file.put("scripts/mod5.py", "@sf_int_stg", auto_compress=False, overwrite=True)

[PutResult(source='mod5.py', target='mod5.py', source_size=41, target_size=48, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [66]:
df = session.sql("list @sf_int_stg")
df.filter(col('"name"').like('%mod5%')).show()

-----------------------------------------------------------------------------------------------------
|"name"                 |"size"  |"md5"                             |"last_modified"                |
-----------------------------------------------------------------------------------------------------
|sf_int_stg/mod5.py     |48      |ba4ad1a67d9fc52ad3096b96398ffd79  |Sat, 16 Dec 2023 17:52:20 GMT  |
|sf_int_stg/mod5.py.gz  |80      |913032dfd39b788b6b66aefc065bd11c  |Sat, 16 Dec 2023 17:52:03 GMT  |
-----------------------------------------------------------------------------------------------------



In [68]:
#Creating a UDF from a Python file on an internal stage:
mod5_udf = session.udf.register_from_file(
     file_path="@sf_int_stg/mod5.py"
    ,func_name="mod5"
    ,return_type=IntegerType()
    ,input_types=[IntegerType()]
    ,replace=True
)

In [69]:
session.range(1,8,2).select(col("ID"), mod5_udf(col("id")).alias("res")).show()

----------------
|"ID"  |"RES"  |
----------------
|1     |1      |
|3     |3      |
|5     |0      |
|7     |2      |
----------------



`Use cache to read a file once from a stage in a UDF:`

In [20]:
path = '/Users/valar/Documents'
basename = os.path.basename(path)
print(basename)

path = '/Users/valar/'
print(os.path.join(path,'Documents'))

import_dir = sys._xoptions.get("snowflake_import_directory")
print(import_dir)


Documents
/Users/valar/Documents
None


In [6]:
import sys
import os
import cachetools
from snowflake.snowpark.types import StringType
@cachetools.cached(cache={})
def read_file(filename):
    import_dir = sys._xoptions.get("snowflake_import_directory")
    if import_dir:
        with open(os.path.join(import_dir, filename), "r") as f:
            return f.read()

# create a temporary text file for test
temp_file_name = "/tmp/temp.txt"
with open(temp_file_name, "w") as t:
     _ = t.write("snowpark")

session.add_import(temp_file_name)
session.add_packages("cachetools")
concat_file_content_with_str_udf = session.udf.register(
     lambda s: f"{read_file(os.path.basename(temp_file_name))}-{s}",
     return_type=StringType(),
     input_types=[StringType()]
    )

[Row(COL1='snowpark-snowflake'), Row(COL1='snowpark-python')]

In [7]:
df = session.create_dataframe(["snowflake", "python"], schema=["a"])
df.select(concat_file_content_with_str_udf("a")).to_df("col1").collect()

[Row(COL1='snowpark-snowflake'), Row(COL1='snowpark-python')]

In [27]:
os.remove(temp_file_name)
session.clear_imports()

#### `Create a vectorized UDF from a lambda with a max batch size and apply it to a dataframe:`

In [3]:
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import IntegerType, PandasSeriesType, PandasDataFrameType

add_udf1 = udf(lambda series1, series2: series1 + series2
               ,return_type=PandasSeriesType(IntegerType())
               ,input_types=[PandasSeriesType(IntegerType()), PandasSeriesType(IntegerType())]
               ,max_batch_size=20
               )

In [4]:
df = session.create_dataframe([[1,2],[3,4]], schema=["a","b"])
df.select(add_udf1("a","b").alias("add_result")).show()

----------------
|"ADD_RESULT"  |
----------------
|7             |
|3             |
----------------



In [5]:
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import IntegerType, PandasSeriesType, PandasDataFrameType

add_udf2 = udf(lambda df: df[0] + df[1]
               ,return_type=PandasSeriesType(IntegerType())
               ,input_types=[PandasDataFrameType([IntegerType(), IntegerType()])]
               ,max_batch_size=20
               )

In [6]:
df = session.create_dataframe([[1,2],[3,4]], schema=["a","b"])
df.select(add_udf2("a", "b")).to_df("add_result").collect()

[Row(ADD_RESULT=3), Row(ADD_RESULT=7)]

#### Create a vectorized UDF with type hints and apply it to a dataframe:

In [8]:
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import IntegerType, PandasSeries, PandasDataFrameType

@udf
def apply_mod5_udf(series: PandasSeries[int]) -> PandasSeries[int]:
    return series.apply(lambda x: x % 5)

In [10]:
session.range(1,8,2).select(apply_mod5_udf("id")).to_df("col").show()

---------
|"COL"  |
---------
|3      |
|1      |
|2      |
|0      |
---------



In [11]:
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import IntegerType, PandasSeries, PandasDataFrame

@udf
def mul_udf(df: PandasDataFrame[int, int]) -> PandasSeries[int]:
    return df[0] * df[1]

In [12]:
df = session.create_dataframe([[1,2],[3,4]]).to_df("a","b")
df.select(mul_udf("a","b")).to_df("col1").show()

----------
|"COL1"  |
----------
|12      |
|2       |
----------



#### `Create a vectorized UDF with original pandas types and Snowpark types and apply it to a dataframe:`

In [4]:
from snowflake.snowpark.functions import pandas_udf
from snowflake.snowpark.types import IntegerType
import pandas as pd

def add1(series1: pd.Series, series2: pd.Series) -> pd.Series:
    return series1 + series2

add_udf1 = pandas_udf(add1
                    ,return_type=IntegerType()
                    ,input_types=[IntegerType(), IntegerType()]
)

SnowparkSessionException: (1409): More than one active session is detected. When you call function 'udf' or use decorator '@udf', you must specify the 'session' parameter if you created multiple sessions.Alternatively, you can use 'session.udf.register' to register UDFs

In [16]:
df = session.create_dataframe([[1, 2], [3, 4]]).to_df("a", "b")
df.select(add_udf1("a", "b")).to_df("add_result").show()

----------------
|"ADD_RESULT"  |
----------------
|3             |
|7             |
----------------



In [3]:
def add2(df: pd.DataFrame) -> pd.Series:
    return df[0] + df[1]

add_udf2 = pandas_udf(add2
                     ,return_type=IntegerType()
                     ,input_types=[IntegerType(), IntegerType()])

NameError: name 'pandas_udf' is not defined

In [18]:
df.select(add_udf2("a", "b")).to_df("add_result").show()

----------------
|"ADD_RESULT"  |
----------------
|7             |
|3             |
----------------



In [2]:
# Close Snowpark session
session.close()